In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import time

In [ ]:
#Tc = 3.640957 #треугольная решетка
Tc = 2.0/np.log(1.+2**0.5) #квадратная решетка

In [ ]:
#Инициализация необходимых рофланов
size = 10
T_ex = 1.0
step = 0.05
T = T_ex - step
T_end = 3.5
i = 0
number = 60000
matrix = np.random.choice([-1, 1], size=(size, size))
sequence = np.arange(T, T_end + step, step)
d_data = {}
energy_data = []
temperatures = np.arange(T_ex, T_end + step, step)

In [ ]:
#Функция перевода сырой конфигурации в корреляционную конфигурацию
def Corel(matrix):
    matrixCorel = np.zeros((matrix.shape[0], matrix.shape[0]))
    for i in range(matrix.shape[0]): 
        for j in range(matrix.shape[0]): 
            L_2_i = int((i + matrix.shape[0]/2) % matrix.shape[0])
            L_2_j = int((j + matrix.shape[0]/2) % matrix.shape[0])
            L_i = i % matrix.shape[0]
            L_j = j % matrix.shape[0]
            matrixCorel[L_i][L_j] = (matrix[L_i][L_j] * matrix[L_2_i][L_j] + matrix[L_i][L_j] * matrix[L_i][L_2_j]) / 2
    return matrixCorel

In [ ]:
#Функция расчета энергии
def ecalc(mas):
    summ = 0
    for i in range(len(mas)):
        for j in range(len(mas) - 1):
            summ += mas[i][j] * mas[i][j + 1] + mas.T[i][j] * mas.T[i][j + 1]
        summ += mas[i][0] * mas[i][len(mas) - 1] + mas.T[i][0] * mas.T[i][len(mas.T) - 1]

    return -summ


In [ ]:
# Функция расчета энергии для треугольной решетки
def ecalc_triangular(mas):
    summ = 0
    for i in range(size):
        for j in range(size):
            summ += mas[i][j] * (mas[i][(j + 1) % size] +
                                 mas[(i + 1) % size][j] +
                                 mas[(i + 1) % size][(j + 1) % size] +
                                 mas[i][(j - 1) % size] +
                                 mas[(i - 1) % size][j] +
                                 mas[(i - 1) % size][(j - 1) % size])
    return -summ / 2 


In [ ]:
#Алгоритм Метрополиса
def mcstep(mas, temper):
    seed1 = random.randint(0, size - 1)
    seed2 = random.randint(0, size - 1)
    energy = ecalc(mas)
    # energy = ecalc_triangular(mas) # для получения данных на треугольной решетке

    mas[seed1][seed2] *= -1
    delta = ecalc(mas) - energy
    # delta = ecalc_triangular(mas)

    if (delta) <= 0:
        pass
    else:
        W = np.exp((-delta) / temper)
        P = random.uniform(0, 1)
        if P > W:
            mas[seed1][seed2] *= -1
            pass
        else:
            pass


In [ ]:
for T in sequence:
    T_round = round(T, 5)
    i = 0
    lattices = []
    total_energy = 0
    energy = 0
    while (i != number):
        mcstep(matrix, T_round)
        matrixCorel = Corel(np.copy(matrix))
        lattices.append(np.copy(matrixCorel))
        energy += ecalc(np.copy(matrix), T_round)
        i += 1
    total_energy = energy / number
    matrix = np.random.choice([-1, 1], size=(size, size))
    if T >= T_ex:
        energy_data.append(total_energy)
        d_data[T_round] = lattices

In [ ]:
plt.plot(temperatures, energy_data, marker='o', linestyle='-')

plt.title('Зависимость энергии от температуры')
plt.xlabel('Температура')
plt.ylabel('Энергия')
plt.grid(True)

plt.show()


In [ ]:
def create_dataset(d_data, Tc, train_ratio=0.7):
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    for T, configs in d_data.items():
        for config in configs:
            if T < Tc:
                label = [1, 0]  # ферромагнитная фаза
            else:
                label = [0, 1]  # парамагнитная фаза

            if np.random.rand() < train_ratio:
                train_data.append(config)
                train_labels.append(label)
            else:
                test_data.append(config)
                test_labels.append(label)

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

In [ ]:
input_shape = (size, size)
hidden_units = 100
output_units = 2 

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(hidden_units, activation='sigmoid'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(output_units, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_data, train_labels, test_data, test_labels = create_dataset(d_data, Tc)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(train_data, train_labels, epochs=32, batch_size=size*size, validation_data=(test_data, test_labels), callbacks=[early_stopping])

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

# Исследование 

In [ ]:
start_time = time.time()

output_layers = []
for T, configs in d_data.items():
    configs_flattened = np.concatenate(configs, axis=0).reshape(-1, size, size)
    outputs = model.predict(configs_flattened)
    average_output_layer = np.mean(outputs, axis=0)
    output_layers.append(average_output_layer)
end_time = time.time()

In [ ]:
time_ = end_time - start_time
print(f"Время выполнения: {time_:.6f} секунд")

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(temperatures, averaged_data, 'x')

plt.axvline(x=Tc, color='red', linestyle='--')

plt.xlabel('Температура')
plt.ylabel('Выходной слой')

plt.legend(['T < Tc', 'T > Tc', 'Tc'])

plt.show()
